# Import Required Libraries
Import the necessary libraries, including TensorFlow, NumPy, and Pandas.

In [1]:
# Import Required Libraries
import tensorflow as tf
import numpy as np
import pandas as pd

# Load and Preprocess Data
Load the stock price data, preprocess it by normalizing and creating sequences for training.

In [2]:
from venv import create
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

# Load the stock price data
data = pd.read_csv('./merged_scaled_data (1).csv')
target = "Q avg to Q avg Price % Change"
# Normalize the data
new_data = data[["Period Ending",target]]
#drop nan values
#having the nan values in the rows affect the traiing as the loss can not be calcluated
#so we drop the rows with nan values
new_data = new_data.dropna()
data = data.dropna()
# Create sequences for training
def create_sequences(data, seq_length, target_column,exclude_columns=None):
    """
    Creates sequences of data for LSTM training.

    Args:
        data (pd.DataFrame): The dataframe containing features and target.
        seq_length (int): The number of timesteps in each sequence.
        target_column (str): The name of the target column.
        exclude_columns (list, optional): List of column names to exclude from features.

    Returns:
        np.array: Array of feature sequences.
        np.array: Array of target values.
    """
    x = []
    y = []
    for i in range(len(data) - seq_length):
        # Define the start and end indices for the sequence
        start_idx = i
        end_idx = i + seq_length

        # Select the sequence slice
        seq_slice = data.iloc[start_idx:end_idx]

        # Exclude specified columns and the target column
        if exclude_columns:
            feature_seq = seq_slice.drop(columns=exclude_columns).values
        else:
            feature_seq = seq_slice.drop(columns=[target_column]).values

        x.append(feature_seq)

        # Select the target value following the sequence
        target_val = data.iloc[end_idx][target_column]
        y.append(target_val)

    return np.array(x), np.array(y)
def create_sequences2(data, seq_length):
    x = []
    y = []
    if len(data) <= seq_length:
        raise ValueError("The length of features must be greater than the sequence length.")
    for i in range(len(data) - seq_length):
        x.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(x), np.array(y)
seq_length = 60
columns_to_drop = ["Ticker Symbol","Period Ending","Sector"]
data = data.drop(columns=columns_to_drop)
x, y = create_sequences(data, seq_length,target)
# x, y = create_sequences2(new_data, seq_length)
# x, y = create_sequences(new_data1,seq_length,target)

# Split the data into training and validation sets
split = int(0.8 * len(x))
x_train, x_val = x[:split], x[split:]
y_train, y_val = y[:split], y[split:]



In [3]:

# Check for NaN or Inf values
def check_nan_inf(array, name):
    print(f"\nChecking {name}:")
    print(f"Number of NaNs: {np.isnan(array).sum()}")
    print(f"Number of Infs: {np.isinf(array).sum()}")
print(x_train.dtype)  # Should be float32 or float64
print(y_train.dtype)  # Should be float32 or float64
print(x_val.dtype)
print(y_val.dtype)
print(x_train.shape)  # Should be (num_samples, seq_length, num_features)
check_nan_inf(x_train, "x_train")
check_nan_inf(y_train, "y_train")
check_nan_inf(x_val, "x_val")
check_nan_inf(y_val, "y_val")

float64
float64
float64
float64
(983, 60, 80)

Checking x_train:
Number of NaNs: 0
Number of Infs: 0

Checking y_train:
Number of NaNs: 0
Number of Infs: 0

Checking x_val:
Number of NaNs: 0
Number of Infs: 0

Checking y_val:
Number of NaNs: 0
Number of Infs: 0


# Build the LSTM Model
Define the LSTM model architecture using TensorFlow's Keras API.

In [4]:
# Build the LSTM Model
units = 25
num_features = x_train.shape[2]
print(x_train.shape[1])
activation = 'tanh'
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(units=units, return_sequences=True, input_shape=(None, num_features)),
    tf.keras.layers.LSTM(units=units),
    tf.keras.layers.Dense(1,activation=activation)
])

model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()
# Train the model
print(x_train.shape)
print(y_train.shape)
print(x_val.shape)
print(y_val.shape)
history = model.fit(
    x_train, 
    y_train, 
    epochs=20, 
    batch_size=64, 
    validation_data=(x_val, y_val))

# Evaluate the model
loss = model.evaluate(x_val, y_val)
print(f'Validation Loss: {loss}')

60


c:\Users\erich\OneDrive\Documents\GitHub\stock_price_prediction\.venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, None, 25)       │        10,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 25)             │         5,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            26 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 15,726 (61.43 KB)

 Trainable params: 15,726 (61.43 KB)

 Non-trainable params: 0 (0.00 B)

(983, 60, 80)
(983,)
(246, 60, 80)
(246,)
Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 44ms/step - loss: 100.6100 - val_loss: 120.2863
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 101.4537 - val_loss: 119.4613
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 99.6966 - val_loss: 119.3375
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 104.9524 - val_loss: 119.3023
Epoch 5/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 99.2847 - val_loss: 119.2885
Epoch 6/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 103.5024 - val_loss: 119.2803
Epoch 7/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 107.3924 - val_loss: 119.2783
Epoch 8/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 103.4060 - val_loss: 119.2763
Epoch 9/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 88.6785 - val_loss: 119.2720
Epoch 10/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 101.3375 - val_loss: 119.2713
Epoch 11/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 101.8763 -

# Compile and Train the Model
Compile the model with an appropriate optimizer and loss function, then train it on the preprocessed data.

In [5]:
# Compile and Train the Model

# Compile the model with an appropriate optimizer and loss function
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model on the preprocessed data
history = model.fit(x_train, y_train, epochs=20, batch_size=32, validation_data=(x_val, y_val))

# Evaluate the model
loss = model.evaluate(x_val, y_val)
print(f'Validation Loss: {loss}')

Epoch 1/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - loss: 100.6195 - val_loss: 120.7476
Epoch 2/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 105.5458 - val_loss: 120.4437
Epoch 3/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 90.8928 - val_loss: 121.8685
Epoch 4/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 99.6185 - val_loss: 121.1315
Epoch 5/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 94.7540 - val_loss: 122.4817
Epoch 6/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 95.9301 - val_loss: 120.7607
Epoch 7/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 94.0913 - val_loss: 121.4169
Epoch 8/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 94.2993 - val_loss: 122.2109
Epoch 9/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 97.9225 - val_loss: 121.4043
Epoch 10/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 104.3047 - val_loss: 121.6292
Epoch 11/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 92.1460 - val_loss: 121.5006
Epoch 12/20
31/31 ━━━━━━━━━

# Evaluate the Model
Evaluate the model's performance on a test dataset.

In [ ]:
# Evaluate the Model
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
# Evaluate the model's performance on the test dataset
loss = model.evaluate(x_val, y_val)
print(f'Validation Loss: {loss}')

# Make predictions on the validation set
predictions = model.predict(x_val)

# Inverse transform the predictions and actual values to get the original scale
actual = y_val.reshape(-1, 1)
mse_val = mean_squared_error(actual, predictions)
print(f'Mean Squared Error: {mse_val}')
root_mse_val = np.sqrt(mse_val)
print(f'Root Mean Squared Error: {root_mse_val}')
# had used r2 score before but for time series data it is not really that good for evaluation because idata in time series exhibits autocorrelation
r2 = r2_score(actual, predictions)
print(f'R^2 Score: {r2}')
##### calculuating the average differences between actual and predicted values
diff = np.abs(actual - predictions)
avg_diff = np.mean(diff)
print(f'Average Difference: {avg_diff}')
# Plot the predictions against the actual values
import matplotlib.pyplot as plt

plt.figure(figsize=(14, 5))
plt.plot(actual, color='blue', label='Actual Stock Price Percentage Change')
plt.plot(predictions, color='red', label='Predicted Stock Price Precentage Change')
plt.title('Stock Price Prediction')
# time is the 250 differnt time stamps from 2012- 2016
plt.xlabel('Time')
plt.ylabel('Stock Price Percentage Change')
plt.legend()
plt.show()

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 117.4300 
Validation Loss: 122.24947357177734
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Mean Squared Error: 8.293243565460802
R^2 Score: -0.06563327492127602


ModuleNotFoundError: No module named 'matplotlib'